In [1]:
# Imports para web scrapping e gravação dos dados coletados

import re, time, os
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import requests
import pandas as pd
import sqlite3
import datetime

In [2]:
# Conecta com banco
# Se o banco não existir, essa mesma função irá criá-lo

conn = sqlite3.connect('houseprices.db')
cur = conn.cursor()

In [3]:
# Função de criar tabela no sqlite

def create_table():
    if True:
        try:
            cur.execute('create table '+table_name+' ('\
                        'data_consulta TEXT, '\
                        'titulo TEXT, '\
                        'endereco TEXT, '\
                        'tipo_imovel TEXT, '\
                        'area TEXT, '\
                        'quarto TEXT, '\
                        'banheiro TEXT, '\
                        'vagas_garagem TEXT, '\
                        'valor TEXT, '\
                        'anunciante TEXT, '\
                        'link TEXT) ')
            print('A tabela ['+table_name+'] foi criada com sucesso!')
        except:
            print('Erro: Não foi possível criar a tabela ['+table_name+']!')

In [4]:
# Executa o chromedriver

chromedriver = "./chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)


In [5]:
# link de referência para a coleta

link = f'https://www.vivareal.com.br/venda/sp/sao-paulo/zona-sul/ipiranga/casa_residencial/?pagina=1'

driver.get(link)
time.sleep(10)

# Aceita cos cookies para continuar a navegação

try:
    driver.find_element_by_class_name('cookie-notifier__cta').click()
except:
    pass

time.sleep(15)

In [6]:
# Converte o formato da página

data_pagina = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
pagina_encode = BeautifulSoup(data_pagina.encode('utf-8'), "lxml")

# Informa quantas casas foram encontradas
# Essa informação pode ser usada como referência ao solicitar a quantidade de dados que deseja coletar

qt_casas = pagina_encode.find('strong', class_ = 'js-total-records')
print('Total de casas encontradas:',qt_casas.text)

Total de casas encontradas:  1.577 


In [7]:
# Função para extrair as informações necessárias

def coleta_dados(casa):
    try:
        # Coleta o título
        q_titulo = casa.find(class_='property-card__title js-cardLink js-card-title').text.strip()
        titulo = q_titulo

        # Coleta endereço
        endereço_completo=casa.find(class_="property-card__address").text.strip()
        endereco = endereço_completo.replace('\n', '')

        # Coleta o tipo de imóvel

        q_tipo_imovel = casa.find(class_='property-card__title js-cardLink js-card-title').text.split()[0]
        q_tipo_imovel = q_tipo_imovel.replace(' ','')\
                                     .replace('\n','')
        tipo_imovel = q_tipo_imovel

        
        # Coleta a área  
        q_area = casa.find(class_="property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area").text.strip()
        area = q_area
        
        # Coleta a quantidade de quartos 

        q_quarto = casa.find(class_="property-card__detail-item property-card__detail-room js-property-detail-rooms").text.strip()
        q_quarto = q_quarto.replace(' ','')\
                           .replace('\n','')\
                           .replace('Quartos','')\
                           .replace('Quarto','')
        quarto = q_quarto

        # Coleta a quantidade de banheiros 

        q_banheiro = casa.find(class_="property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom").text.strip()        
        q_banheiro = q_banheiro.replace(' ','')\
                               .replace('\n','')\
                               .replace('Banheiros','')\
                               .replace('Banheiro','')
        banheiro = q_banheiro 

        # Coleta numero de vagas de garagem
        q_vagas_garagem=casa.find(class_="property-card__detail-item property-card__detail-garage js-property-detail-garages").text.strip()        
        q_vagas_garagem=q_vagas_garagem.replace(' ','')\
                                       .replace('\n','')\
                                       .replace('Vagas','')\
                                       .replace('Vaga','')
        vagas_garagem = q_vagas_garagem

        # Coleta preço da casa

        q_valor = q_valor = casa.find(class_="property-card__price js-property-card-prices js-property-card__price-small").text.strip()
        valor = q_valor
        
        # Coleta o link
        q_link = casa.find(class_='property-card__main-info').a.get('href')
        link = q_link
        
        # Coleta o anunciante
        q_anunciante = casa.find(class_='property-card__account-link js-property-card-account-link').img.get('alt').title()
        anunciante = q_anunciante
        
        data_atual = str(datetime.datetime.now().strftime('%Y-%m-%d'))
        
        # Gera uma lista com os dados extraídos
        
        linha_coleta = [data_atual,titulo,endereco,tipo_imovel,area,quarto,banheiro,vagas_garagem,valor,anunciante,link]
        
        return linha_coleta
    except:
        pass

In [8]:
%%time

# Pergunta quantas casas serão pesquisadas

total_pesquisa = int(input('Quantas casas você deseja pesquisar? '))

# Pergunta o nome da tabela que será criada para guardar os dados coletados

table_name = input('Digite o nome da tabela: ')

# inicia o contador

count = 1

# cria a tabela

create_table()

# Função ler os containers e trocar de página sempre que for necessário

try:
    while count <= total_pesquisa:
        
        # inicia o totalizador de containers
        total_linhas = 0
        
        # coleta todas as informações da página e transforma em um formato legivel
        data_pagina = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
        pagina_encode = BeautifulSoup(data_pagina.encode('utf-8'), "lxml")

        # identifica todos os containers de imóveis
        pagina = pagina_encode.find(class_='results-list js-results-list')

        # loop para contar quantidade containers na página
        for linha_casa in pagina.findAll(class_="property-card__container js-property-card"):
            total_linhas += 1

        # inicia o contador de container
        count_linha = 0
        
        # Loop para coletar os dados coletados na página
        while count_linha < total_linhas:
            # Loop para coletar os dados de cada container
            for casa in pagina.findAll(class_="property-card__container js-property-card"):
                if count <= total_pesquisa:
                    coleta = coleta_dados(casa)
                    
                    cur.execute('insert into '+table_name+' values '\
                                '(?,?,?,?,?,?,?,?,?,?,?)',coleta)


                count_linha += 1
                count += 1

        # Aguarda 20 segundos antes de trocar de página 
        time.sleep(20)

        receita = driver.find_element_by_xpath(f'//*[@id="js-site-main"]/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[9]/a')
        receita.click()
except:
    pass

# Commit do SQLite 
conn.commit()    

# fecha o chromedriver
driver.quit()

print('Trabalho concluído!')

Quantas casas você deseja pesquisar?  1577
Digite o nome da tabela:  houseprice_vivareal


A tabela [houseprice_vivareal] foi criada com sucesso!
Trabalho concluído!
Wall time: 19min 21s
